# 2-Opt and 3-Opt CVRP

### Разделение тура на подтуры с учетом весов. Рассчет длины всего маршрута. Использование алгоритма "Ближайший сосед" для оптимизации начального тура

In [4]:
# Получаем список городов, на которых суммарный вес превышает вместимость грузовика
def subtourslice(tour, vehicle, max_vehicle):
    #print("--------------------------------Начало работы функции--------------------------------")
    #vehilce[i][0] - номер города, vehicle[i][1] - вес груза для перевозки
    capacity_used = np.zeros(len(vehicle))
    k = 0
    slice = []
    mass = [] # список всех весов грузов для перевозки
    for i in range(len(vehicle)):
        # capacity_used[i] - начальный вес
        while((capacity_used[i] <= max_vehicle) and (vehicle[k][0] <= (len(tour) - 3))):
            capacity_used[i] += vehicle[k][1] # заполняем грузовик конкретным кол-вом груза для каждого города
            if(capacity_used[i] > max_vehicle):
                # если кол-во груза, который нужно отвезти, больше чем вместимость грузовика, то вычитаем это кол-во груза
                capacity_used[i] -= vehicle[k][1]
                slice.append(vehicle[k-1][0]) # запоминаем номер тура, чтобы повторно не проходить по такому же маршруту
                break
            k += 1
    slice.append(vehicle[k-1][0])
    return(slice)

def subtour(slice, tour):
    sub = []
    # добавляем номера городов, после которых суммарный вес груза превышал объем грузовика
    sub.append(tour[:(slice[0] + 1)])
    for i in range(0, len(slice) - 1):
        # нарежем маршруты городов до момента, пока суммарный вес груза не превысел объем грузовика
        sub.append(tour[(slice[i] + 1):(slice[i + 1] + 1)])
    return sub

def total_distance_tour(tour, distance):
    d = 0
    for n in range(0, len(tour)):
        if(len(tour[n]) == 1):
            d = d + 2 * distance[(tour[n][0], 0)]
        else:
            for p in range(1, len(tour[n])):
                i = tour[n][p-1]
                j = tour[n][p]
                d = d + distance[(i, j)]

            i = tour[n][len(tour[n]) - 1]
            d = d + distance[(i, 0)]


            i = tour[n][1]
            d = d + distance[(i, 0)]
    return d

def all_vechile_distance(sub, distance):
    all_distance = total_distance_tour(sub, distance)
    return all_distance

def tour_to_distance(tour, cities, distance, vehicle, max_vehicle):
    u = subtourslice(tour, vehicle, max_vehicle) # получаем спислок городов, после которых суммарный вес груза > объема грузовика
    v = subtour(u, tour) # получим тур из номеров городов, где u[0] это список из номеров городов, после которых суммарный вес груза > объема грузовика
    total = all_vechile_distance(v, distance) # посчитаем суммарное расстояние между всеми городами в туре
    return total

def Nearest_Neighbour(starting_node, cities, distance, q, max_vehicle):
    #Поставим стартовый узел и добавим его в список узлов
    NN = [starting_node]
    n = len(cities)
    
    while(len(NN) < n):
        k = NN[-1] # берём последний узел
        # считаем от этого узла расстояние до всевозможных остальных узлов и берем минимальное
        nn_len = []
        nn_idx = []
        for j in cities:
            if((k!=j) and (j not in NN)):
                nn_idx.append((k, j))
                nn_len.append(distance[(k, j)])
        min_idx = nn_idx[0][1]
        min_len = nn_len[0]
        for i in range(0, len(nn_len)-1):
            if(nn_len[i] < min_len):
                if(q[nn_idx[i][0]][1] + q[nn_idx[i][1]][1] < max_vehicle):
                    min_len = nn_len[i]
                    min_idx = nn_idx[i][1]
        NN.append(min_idx) # запоминае новый узел
    
    cost = tour_to_distance(NN, cities, distance, q, max_vehicle)
    return NN, cost

## Algorithm 2-Opt

In [5]:
def Two_Opt(NN, distance):
    min_change = 0
    change = 0
    min_i = 0
    min_j = 0
    for i in range(len(NN) - 2):
        for j in range(i+2, len(NN)-1):
            cost_actual = distance[(NN[i], NN[i+1])] + distance[(NN[j], NN[j+1])]
            #Убираем 2 ребра и вставляем 2 новых
            cost_new = distance[(NN[i], NN[j])] + distance[(NN[i+1], NN[j+1])]
            #Далее проверяем улучшился маршрут после перестановки или нет
            change = cost_new - cost_actual
            if(change < min_change):
                min_change = change
                #Также запомни индексы, на которых при перестановки маршрут улучшился
                min_i = i
                min_j = j

    if(min_change < 0):
        NN[min_i+1:min_j+1] = NN[min_i+1:min_j+1][::-1] # переворачиваем целый список городов
    return NN


## Algorithm 3-Opt

In [6]:
def SwapThree(tour, a, c, e, cities, distance, q, max_vehicle):
    """
    Функция: SwapThree
     
    Описание:  
    
    Эта функция воссоздает маршрут, отсоединяя и повторно соединяя 3 ребра ab, cd и ef 
    (таким образом, чтобы результат все еще был полным и выполнимым туром).
    
    Эта функция учитывает только чистые 3-Opt ходы. Никакие 2-Opt ходы не допускаются.
    
    
   (a)           - Позиция первого узла в списке
   (c)           - Позиция второго узла в списке
   (e)           - Позиция третьего узла в списке
    
    """    
    # узлы сортируются для обеспечения более простой реализации
    a, c, e = sorted([a, c, e])
    b, d, f = a+1, c+1, e+1
    
    new_tour_1 = list()
    new_tour_2 = list()
    new_tour_3 = list()
    new_tour_4 = list()
    
    # рассматриваются четыре различных соединения туров
    new_tour_1 = tour[:a+1] + tour[c:b-1:-1] + tour[e:d-1:-1] + tour[f:] # 3-opt
    new_tour_2 = tour[:a+1] + tour[d:e+1]    + tour[b:c+1]    + tour[f:] # 3-opt
    new_tour_3 = tour[:a+1] + tour[d:e+1]    + tour[c:b-1:-1] + tour[f:] # 3-opt
    new_tour_4 = tour[:a+1] + tour[e:d-1:-1] + tour[b:c+1]    + tour[f:] # 3-opt
    
    len_1 = tour_to_distance(new_tour_1, cities, distance, q, max_vehicle)
    len_2 = tour_to_distance(new_tour_2, cities, distance, q, max_vehicle)
    len_3 = tour_to_distance(new_tour_3, cities, distance, q, max_vehicle)
    len_4 = tour_to_distance(new_tour_4, cities, distance, q, max_vehicle)
    
    best_move = min(len_1,len_2,len_3,len_4)
    if(best_move == len_1):
        return new_tour_1
    elif(best_move == len_2):
        return new_tour_2
    elif(best_move == len_3):
        return new_tour_3
    elif(best_move == len_4):
        return new_tour_4

def Three_opt(tour, dist, cities, q, max_vehicle):
    """
    Функция: ThreeOpt
     
    Описание: 
    
    Эта функция применяет алгоритм 3-Opt для поиска нового тура с более низкой стоимостью, чем входной тур.
    
    Алгоритм сканирует все узлы a, c, e и меняет местами три ребра, соединяющие текущий тур. 
    Предпринимаются попытки всех четырех различных пересоединений трех ребер, и алгоритм останавливается при первом улучшении.
    
    Если обнаружено улучшение, тур меняется местами, и новый тур используется для оценки дальнейших улучшений.
    
    Алгоритм останавливается, когда дальнейшее улучшение не может быть найдено путем замены трех ребер с учетом одной из 
    4 возможностей.
    
    """
    size = len(tour)
    new_tour = []
    old_tour = tour
    best_cost = tour_to_distance(old_tour, cities, distance, q, max_vehicle)
    improve = 0
    while(improve <= 0):        
        for a in range(1,size-2):
            for c in range(a+1,size-1):
                for e in range(c+1,size):
                        new_tour = SwapThree(tour, a, c, e, cities, distance, q, max_vehicle) 
                        new_cost = tour_to_distance(new_tour, cities, distance, q, max_vehicle)
#                         print("NEW COST", new_cost)
#                         print("BEST COST", best_cost)
                        if (new_cost < best_cost):
                            tour = new_tour
                            best_cost = new_cost
                            improve = 0

        improve += 1
    return tour

In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import operator
from functools import reduce
import itertools
import random

n = 51
cities = [i for i in range(n)] # список номеров городов
print(cities)

roads = [(i, j) for i in range(n) for j in range(n) if(i != j)] # это попарные маршруты между всеми городами

x = np.random.rand(n) * 10
y = np.random.rand(n) * 10


distance = {(i,j):np.hypot(x[i] - x[j], y[i]-y[j]) for i,j in roads}
N = [i for i in range(1, n)] # tour без депо

min_capasity = 1
max_capasity = 42
max_vehicle = 40
capacity = np.minimum(np.maximum(np.abs(np.random.normal(15, 10, size=[1, n])), min_capasity), max_capasity)

p = 1
max_vehicle *= p
while(max_vehicle < max_capasity):
    max_vehicle = max_vehicle / p
    p += 1
    max_vehicle *= p
print("Используется грузовиков: ", p)

q = []
for i in range(1, len(N)+1):
    q.append((i, capacity[0][i])) # вес груза, который необходимо перевести в город

time_start = time.time()

starting_node = 0
NN, start_cost = Nearest_Neighbour(starting_node, cities, distance, q, max_vehicle) # Создаем начальный тур, который уже каким-то образом оптимизирован
solution1 = NN.copy()
print("Start solution", NN)

flag = 1 
k1 = 0 # кол-во шагов
print("Start distance", start_cost)
cost = start_cost
while(flag != 0):
    k1 += 1
    solution1 = Two_Opt(solution1, distance).copy()
    new_cost = tour_to_distance(solution1, cities, distance, q, max_vehicle)
    flag = np.abs(new_cost - cost)
    cost = new_cost

starting_node = 0
flag = 1 
k2 = 0 # кол-во шагов
NN, start_cost = Nearest_Neighbour(starting_node, cities, distance, q, max_vehicle) # Создаем начальный тур, который уже каким-то образом оптимизирован
solution2 = NN.copy()
cost = start_cost
while(flag > 0):
    k2 += 1
    solution1 = Three_opt(solution1, distance, cities, q, max_vehicle).copy()
    new_cost = tour_to_distance(solution1, cities, distance, q, max_vehicle)
    flag = np.abs(new_cost - cost)
    cost = new_cost
time_final = time.time()

print(" ")
print("Result Two-Opt")
print("Optimize solution", solution1)
print("Optimize distance", cost)
print("Steps", k1)
print(" ")
print("Result Three-Opt")
print("Optimize solution", solution2)
print("Optimize distance", cost)
print("Work time", time_final - time_start)
print("Steps", k2)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50]
Используется грузовиков:  2
Start solution [0, 32, 13, 36, 9, 46, 27, 5, 8, 42, 41, 17, 22, 48, 47, 40, 24, 14, 7, 20, 16, 31, 12, 11, 49, 26, 35, 19, 28, 30, 4, 34, 39, 38, 33, 44, 37, 25, 6, 2, 29, 21, 3, 18, 45, 1, 15, 23, 43, 10, 50]
Start distance 148.6106497440547
 
Result Two-Opt
Optimize solution [0, 32, 13, 30, 21, 3, 18, 45, 9, 46, 27, 5, 1, 15, 23, 36, 29, 33, 38, 39, 34, 28, 4, 19, 35, 26, 7, 14, 40, 24, 16, 20, 31, 12, 11, 6, 2, 25, 37, 44, 49, 47, 48, 22, 17, 41, 50, 42, 8, 43, 10]
Optimize distance 129.84002586031184
Steps 13
 
Result Three-Opt
Optimize solution [0, 32, 13, 36, 9, 46, 27, 5, 8, 42, 41, 17, 22, 48, 47, 40, 24, 14, 7, 20, 16, 31, 12, 11, 49, 26, 35, 19, 28, 30, 4, 34, 39, 38, 33, 44, 37, 25, 6, 2, 29, 21, 3, 18, 45, 1, 15, 23, 43, 10, 50]
Optimize distance 129.8400